In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw/credit_record.csv')
df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [3]:
df.dtypes

ID                 int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

In [4]:
print(f'Rows: {df.shape[0]} \nColumns: {df.shape[1]}')

Rows: 1048575 
Columns: 3


In [5]:
print(f'Null values: {df.isna().sum().sum()}')
print(f'Duplicated values: {df.duplicated().sum()}')

Null values: 0
Duplicated values: 0


In [6]:
random_id = df.sample(1).ID.values[0]

In [7]:
df[df.ID == random_id]

,ID,MONTHS_BALANCE,STATUS
816626,5116402,0,C
816627,5116402,-1,C
816628,5116402,-2,C
816629,5116402,-3,C
816630,5116402,-4,C
816631,5116402,-5,C
816632,5116402,-6,C
816633,5116402,-7,C
816634,5116402,-8,C
816635,5116402,-9,C


In [8]:
df[['STATUS']].value_counts()

STATUS
C         442031
0         383120
X         209230
1          11090
5           1693
2            868
3            320
4            223
Name: count, dtype: int64

In [9]:
df_g = df.groupby(['ID','STATUS'])
df_g = df_g.size().unstack()
df_g.head()

STATUS,0,1,2,3,4,5,C,X
ID,,,,,,,,
5001711,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5001712,10.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
5001715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0


In [10]:
df_g['total'] = df_g.sum(axis=1)

bad = ['0', '1', '2', '3', '4', '5']
good = ['C', 'X']

df_g['bad'] = df_g[bad].sum(axis=1)
df_g['good'] = df_g[good].sum(axis=1)

df_g['good_ratio'] = df_g['good'] / df_g['total']

df_g[['bad', 'good', 'total', 'good_ratio']].describe()

STATUS,bad,good,total,good_ratio
count,45985.000000,45985.000000,45985.000000,45985.000000
mean,8.640078,14.162466,22.802544,0.524158
std,8.507858,14.711766,15.492771,0.354110
min,0.000000,0.000000,1.000000,0.000000
25%,3.000000,1.000000,10.000000,0.166667
50%,6.000000,9.000000,19.000000,0.571429
75%,12.000000,23.000000,34.000000,0.844828
max,61.000000,61.000000,61.000000,1.000000


In [11]:
df_g['CLASS'] = np.where(df_g['good_ratio'] >= 0.7, 1, 0)
df_g.sample(3)

STATUS,0,1,2,3,4,5,C,X,total,bad,good,good_ratio,CLASS
ID,,,,,,,,,,,,,
5052920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,0.0,12.0,1.000000,1
5121282,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,0.0,0.000000,0
5094921,14.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,21.0,14.0,7.0,0.333333,0


In [12]:
df_g = df_g.reset_index(drop=False)[['ID', 'CLASS']]

class_count = dict(df_g[['CLASS']].value_counts())

total = sum(class_count.values())

for key in class_count.keys():
    value = class_count[key]
    perc = (value / total) * 100
    
    print(f'{key[0]}: {value} ({perc:.2f}%)')

0: 27639 (60.10%)
1: 18346 (39.90%)


In [13]:
application = pd.read_csv('../data/raw/application_record.csv')
application.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [14]:
classes = pd.merge(application, df_g, how='inner', on='ID')
classes.to_csv('../data/interim/class_record.csv', index=False)